# <b>Watershed snow line/snow level comparison</b>
## An interactive tool to compare two elevations across a given watershed 
### 9 Dec 2022 KH

#### This tool visualizes two elevations against a watershed's hypsometric curve and maps them. 

Using previously calculated elevation CDFs across the HUC8 basins (see calc_huc8_cdfs.py), 
this notebook simply deploys an interactive widget displaying two elevations across a basin. 

The user is prompted to select a basin and provide both elevations as input. 

The output plots (and interpretively shades) both elevations on the CDF curve and as contours on a map of the basin.

---

Currently, this is intended to integrate into a near real-time snowline monitoring tool, which 
uses elevations of :
* (1) the remotely-sensed snow cover ("snowline", e.g., from MODSCAG), and 
* (2) the forecasted elevation of the 0-degree altitude ("snow level", e.g., from GEFS ensemble mean).

Comparing these elevations can give us a rough sense of where "active" or "passive" snowpack areas may be. Inferring active/passive areas of course assume the precip is uniformly distributed in space and time.

So, the variable definitions and inputs below aim to emulate that functionality

#### The code below does the following:
* (1) Import and preprocess elevation and watershed data (elev .tif, basins .shp, and CDFs .csv files)
* (2) Set up plotting scheme (colors, lines, etc. a workflow for the interactive figure)
* (3) Launch the user interaction

---


### Load libraries

In [1]:
import os
import pandas as pd
import numpy as np
import xarray as xr
import rasterio
import rioxarray as rxr
import geopandas as gpd
import datetime as dt
from shapely.geometry import mapping
import matplotlib.pyplot as plt
%matplotlib inline 

---

---

### 1: Import/process elevation and watershed data


---

#### 1a: Import csv of elevation CDFs

In [2]:
# Import table of elevation CDFs (rows=CDF, columns=HUC8, values=elev)
cdf_fname = 'C:/Users/kden/research/projects/active/cw3e_SnowlineMonitor/huc8_cdfs.csv'
cdf = pd.read_csv(cdf_fname, index_col=0)
cdf.index = np.round(cdf.index, 2)  # <-- cut precision down (e.g., some values were 0.94000000001)
cdf.head(2)

,18100204,18070303,18070304,18020162,18050001,15030102,18100100,18030003,18060003,18060007,...,17010110,18010101,18010206,15070101,17100201,17100202,17090010,17080003,17080006,17090012
CDF,,,,,,,,,,,,,,,,,,,,,
0.00,0.0,0.0,0.0,2.0,0.0,156.00,107.0,82.0,582.0,110.0,...,467.0,0.0,419.0,164.00,0.0,0.0,34.0,0.0,0.0,0.0
0.01,0.0,0.0,0.0,6.0,0.0,390.15,130.0,86.0,585.0,214.0,...,540.0,0.0,564.0,179.32,0.0,11.0,40.0,0.0,0.0,2.0


---

#### 1b: Import DEM (mosaic each MODIS tile into one array)

In [3]:
# Read in and mosaic MODIS DEM
tiles = ['h08v05','h08v04','h09v04','h10v04']
zdir = 'C:/Users/kden/research/DATA/geog/dem500m_modis/tif/'
# loop through tiles
for i in range(len(tiles)):
    itile = tiles[i]
    # locate the filename that matches the tile
    zfname = [x for x in os.listdir(zdir) if itile in x][0]
    z = rxr.open_rasterio(zdir+zfname)
    # "concatenate" each tile
    if i==0:
        dem = z
    else:
        dem = dem.combine_first(z)
dem.data[dem.data==dem._FillValue] = np.nan
# set "straggling" below-0 values to 0
dem.data[dem.data<0] = 0
dem

<xarray.DataArray (band: 1, y: 4800, x: 7200)>
array([[[315., 308., 290., ...,  nan,  nan,  nan],
        [314., 301., 293., ...,  nan,  nan,  nan],
        [352., 340., 328., ...,  nan,  nan,  nan],
        ...,
        [  0.,   0.,   0., ..., 828., 834., 833.],
        [  0.,   0.,   0., ..., 821., 829., 832.],
        [  0.,   0.,   0., ..., 789., 808., 821.]]], dtype=float32)
Coordinates:
  * y            (y) float64 3.336e+06 3.337e+06 ... 5.559e+06 5.56e+06
  * x            (x) float64 -1.112e+07 -1.112e+07 ... -7.784e+06 -7.784e+06
  * band         (band) int32 1
    spatial_ref  int32 0
Attributes:
    AREA_OR_POINT:       Area
    STATISTICS_MAXIMUM:  4290
    STATISTICS_MEAN:     -9999
    STATISTICS_MINIMUM:  -81
    STATISTICS_STDDEV:   -9999
    _FillValue:          -3.4e+38
    scale_factor:        1.0
    add_offset:          0.0
    long_name:           layer

In [4]:
dem.rio.crs

CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["unknown",SPHEROID["unknown",6371007.181,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Sinusoidal"],PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')

---

#### 1c: Import HUC8 basins shapefile (and project them to the MODIS grid)
Reprojecting a vector is computationally cheaper and arguably less distorting than reprojecting the raster

In [5]:
huc8_fname = 'C:/Users/kden/research/projects/active/cw3e_SnowlineMonitor/gis/'
#huc8_fname += 'huc8_wus_v1/huc8_wus_v1.shp'
huc8_fname += 'huc8_coastmtns_v1/huc8_coastmtns_v1.shp'

huc8 = gpd.read_file(huc8_fname)
phuc8 = huc8.to_crs(dem.rio.crs)
# Let's just isolate a few key columns
phuc8 = phuc8[['huc8','name','areasqkm','Shape_Leng', 'Shape_Area', 'geometry']]
phuc8.head(2)

,huc8,name,areasqkm,Shape_Leng,Shape_Area,geometry
0,18070303,San Luis Rey-Escondido,2153.24,3.219530,0.208313,"POLYGON ((-10862697.466 3717882.775, -10862712..."
1,18070304,San Diego,4020.53,3.504593,0.387407,"POLYGON ((-10866552.600 3696499.063, -10866553..."


For now, let's just consider a few HUC8s of interest.

We can later use the entire WUS


In [6]:
my_huc8s = ['18020121',  # NF Feather (FIRO)
            '18020122',  # E branch NF Feather (FIRO)
            '18020123',  # Middle fork Feather (FIRO)
            '18020125',  # Upper Yuba (FIRO)
            '18020128',  # NF American
            '18020129',  # SF American
            
            '18040009',  # Upper Tuolumne
            '18040008'  # Upper Merced
            ]

---

#### 1d: Store a cropped DEM for each basin for later access

To make the widget more efficient, let's frontload the computing cost of masking the DEM to each basin now.

In [7]:
# define a dictionary
my_demdict = {}

# loop through our HUC8s, clip the DEM, and store it in the dictionary
for i in range(phuc8.shape[0]):
    ihuc8id = phuc8['huc8'].iloc[i]
    tmp = dem.rio.clip(phuc8.loc[phuc8['huc8']==ihuc8id].geometry.apply(mapping))
    tmp.data[tmp.data==tmp._FillValue] = np.nan
    tmp.data[tmp.data<0] = 0
    # assign the huc8 id as the clipped DEM "key" 
    my_demdict[ihuc8id] = tmp.squeeze()
    # make progress verbose
    if np.mod(i,25)==0:
        print('Stored %s of %s basin DEMs...' % (i, phuc8.shape[0]))
print('All done')

Stored 0 of 116 basin DEMs...
Stored 25 of 116 basin DEMs...
Stored 50 of 116 basin DEMs...
Stored 75 of 116 basin DEMs...
Stored 100 of 116 basin DEMs...
All done


---

---

### 2: Set up plotting scheme

In [8]:
# Function -- mask a particular region of the DEM between two elevations
def isolate_elevband(demraster, elevmin, elevmax):
    mask = demraster.copy()
    mask.data[mask.data < elevmin] = np.nan
    mask.data[mask.data > elevmax] = np.nan
    mask.data[(mask.data>=elevmin) & (mask.data<=elevmax)] = 1
    return mask

In [9]:
# Function to condition values/colors/plot on input values

def plot_map_cdf(HUC8_id, Snowline, Snow_Level):
    
    ihuc8id = HUC8_id
    irsle = Snowline
    z0c = Snow_Level

    # locate CDF index value of snowline and z0c
    idx_rsle = cdf.index[np.argmin(np.abs(cdf[ihuc8id]-irsle))]
    idx_z0c = cdf.index[np.argmin(np.abs(cdf[ihuc8id]-z0c))]
    zmax = cdf[ihuc8id].values[-1]

    # "snow" color
    csnow = 'C0'
    csnowline = 'b'
    csnowmap = 'Blues'
    # condition z0 color and plot position values based on below/above snowline
    if z0c >= irsle:
        cz0 = 'C3'
        x1 = idx_rsle
        x2 = idx_z0c
        y1 = irsle
        y2 = z0c
        czmap = 'Reds'
    else:
        cz0 = 'C2'
        x1 = idx_z0c
        x2 = idx_rsle
        y1 = z0c
        y2 = irsle
        czmap = 'Greens'

    # grab dem for basin above-snowline contour
    idem = my_demdict[ihuc8id]

    # mask above-snowline and between z0c/snowline areas of the DEM
    idem_snowmask = isolate_elevband(idem, irsle, cdf[ihuc8id].values[-1])
    idem_z0cmask = isolate_elevband(idem, y1, y2)



    # %% set up figure (cdf+map)
    fig,axs=plt.subplots(1,2, figsize=(8,4), tight_layout=True)

    # plot CDF and basin
    cdf[[ihuc8id]].plot(ax=axs[0], c='k', legend=False)
    phuc8.loc[phuc8['huc8']==ihuc8id].plot(ax=axs[1], fc='none', ec='k')

    # % shade in "above-snowline" and add the snowline line

    axs[0].fill_between(x = cdf[ihuc8id].loc[cdf.index>=idx_rsle].index, 
                        y1 = cdf[ihuc8id].loc[cdf.index>=idx_rsle].values, 
                        fc=csnow, ec='none', alpha=0.4)
    axs[0].fill_betweenx(y = cdf[ihuc8id].loc[cdf[ihuc8id]>=irsle].values, 
                         x1 = cdf[ihuc8id].loc[cdf[ihuc8id]>=irsle].index, 
                         fc=csnow, ec='none', alpha=0.4)
    axs[0].plot((0,idx_rsle), (irsle, irsle), ls='--', c=csnowline)
    axs[0].plot((idx_rsle,idx_rsle), (0,irsle), ls='--', c=csnowline)

    # % shade in between-snowlne and z0c

    axs[0].fill_between(x = cdf[ihuc8id].loc[(cdf.index>=x1) & (cdf.index<=x2)].index, 
                        y1 = cdf[ihuc8id].loc[(cdf.index>=x1) & (cdf.index<=x2)].values, 
                        fc=cz0, ec='none', alpha=0.4)
    axs[0].fill_betweenx(y = cdf[ihuc8id].loc[(cdf.index>=x1)&(cdf.index<=x2)].values,
                          x1 = cdf[ihuc8id].loc[(cdf.index>=x1)&(cdf.index<=x2)].index, 
                          fc=cz0, ec='none', alpha=0.4)
    axs[0].plot((0,idx_z0c), (z0c, z0c), ls='--', c=cz0)
    axs[0].plot((idx_z0c,idx_z0c), (0,z0c), ls='--', c=cz0)


    # % map above-snowline and between snowline/z0c
    idem.plot.contour(ax=axs[1], levels=[irsle], colors=(csnowline), alpha=0.4)
    idem_snowmask.plot(ax=axs[1], cmap=csnowmap, add_colorbar=False, alpha=0.7)
    idem.plot.contour(ax=axs[1], levels=[z0c], colors=(cz0), alpha=0.4)
    idem_z0cmask.plot(ax=axs[1], cmap=czmap, add_colorbar=False, alpha=0.7)

    # % configure axes
    [axs[0].spines[side].set_visible(False) for side in ['top','right']]
    axs[1].axis('off')
    axs[1].set_title('')
    axs[0].set_ylabel('Elevation (m)')
    fig.suptitle(phuc8['name'].loc[phuc8['huc8']==ihuc8id].values[0]+'\n'+ihuc8id, y=0.9)
    
    return fig,axs


---

---

### 3: deploy interactive widget

Prompt user to select basin from dropdown menu (it'd be great to have a leaflet map instead)

Then, prompt the snow level / snowline on integer sliders 

In [10]:
from ipywidgets import interact, FloatSlider, widgets, interactive
from IPython.display import display

In [14]:
interact(plot_map_cdf, 
                HUC8_id = widgets.Dropdown(options=my_huc8s, value='18020125'),
                Snowline = widgets.IntSlider(value=1500, min=500, max=4000, step=10), 
                Snow_Level = widgets.IntSlider(value=1200, min=500, max=4000, step=10))

interactive(children=(Dropdown(description='HUC8_id', index=3, options=('18020121', '18020122', '18020123', '1…

<function __main__.plot_map_cdf(HUC8_id, Snowline, Snow_Level)>

In [11]:
w = interactive(plot_map_cdf, 
                HUC8_id = widgets.Dropdown(options=my_huc8s, value='18020125'),
                Snowline = widgets.IntSlider(value=1500, min=500, max=4000, step=10), 
                Snow_Level = widgets.IntSlider(value=1200, min=500, max=4000, step=10))

In [12]:
display(w)

interactive(children=(Dropdown(description='HUC8_id', index=3, options=('18020121', '18020122', '18020123', '1…

### NOTE 
#### the "interactive" module was not what gave the example static interactivity. It was:
from ipywidgets import StaticInteract

Either way, one of our big problems is that converting to HTML fails (jinja / nbconvert incompatible with what we've got?)

^ Solved that with downgrading jinja2 from version 3.1.x to 3.0.3 -- that enabled this notebook to get downloaded with <b>jupyter nbconvert --to html huc8cdf_widget_v0.ipynb</b> in cmd. 

However, the widget images still do not update

---

---

### Test out interactive map using folium

In [13]:
import folium

Available base maps are:
* OpenStreetMap”
* “Mapbox Bright” (Limited levels of zoom for free tiles)
* “Mapbox Control Room” (Limited levels of zoom for free tiles)
* “Stamen” (Terrain, Toner, and Watercolor)
* “Cloudmade” (Must pass API key)
* “Mapbox” (Must pass API key)
* “CartoDB” (positron and dark_matter)

In [14]:
m = folium.Map(location = [39, -120], zoom_start=6
              , tiles = 'Stamen Terrain'
              )

# add polygons
geo_j = huc8.loc[huc8['huc8'].isin(my_huc8s)].to_json()
geo_j = folium.GeoJson(data=geo_j, 
                      style_function=lambda x: {'fillColor':'C1'
                                                #,'fillOpacity':0.2
                                                #,'color':'r'
                                               }, 
                       # tooltip shows fields from the geodataframe when mouse hovers over the shape
                      tooltip=folium.features.GeoJsonTooltip(fields=['name'],
                                                            aliases=[''])
                       # popup shows fields when we click the shape
                       ,popup=folium.features.GeoJsonPopup(fields=['huc8','areasqkm'])
                      )
geo_j.add_to(m)
m

---

---
 #### Can we launch the widget once we click a basin?

I think if we could conjure the above-widget as an HTML, <br>
then we could include it in the map widget as a popup, maybe. <br>
https://gis.stackexchange.com/questions/185897/how-can-i-include-html-in-a-folium-marker-popup